## clf trained by the simple data set

In [125]:
import matplotlib.pyplot as plt
from PIL import Image
import json
import skimage
from skimage.feature import hog
from sklearn import svm

### preparing testing and training data
#### also tryout with slighly different angles for testing in config2.json:
json_string_testing = open("data/data_set_simple/testing/config2.json", "r").read()

In [141]:
def preprocess_input(image):
    # Resize the image
    image = image.resize((165, 165))
    fd =  hog(image, orientations=8, pixels_per_cell=(16, 16), cells_per_block=(1, 1))
    return fd

json_string_training = open("data/data_set_simple_2/training/config.json", "r").read()
data_training = json.loads(json_string_training)
X = []
Y = []
for image_data in data_training:
    # Create the image path
    image_path = "./data/data_set_simple_2/training/{}".format(image_data['path'])
    # Read in the image
    image = Image.open(image_path).convert("L")
    # Append the processed image
    X.append(preprocess_input(image))
    # Append the angle
    Y.append(image_data["angle"])

json_string_testing = open("data/data_set_simple_2/testing/config.json", "r").read()
data_testing = json.loads(json_string_testing)
test_X = []
test_Y = []
for image_data in data_testing:
    # Create the image path
    image_path = "./data/data_set_simple_2/testing/{}".format(image_data['path'])
    # Read in the image
    image = Image.open(image_path).convert("L")
    # Append the processed image
    test_X.append(preprocess_input(image))
    # Append the angle
    test_Y.append(image_data["angle"])

In [142]:
clf = svm.SVC(gamma="scale", decision_function_shape='ovo')
clf_fit = clf.fit(X, Y)

### pickle example

In [143]:
import pickle

In [144]:
file_name = "clf_file.pickle"
file_object = open(file_name,'wb')
pickle.dump(clf_fit,file_object,protocol=2)
file_object.close()

```python
a = ['test value','test value 2','test value 3']

file_name = "testfile.pickle"
# open the file for writing
file_object = open(file_Name,'wb') 

# this writes the object a to the
# file named 'testfile'
pickle.dump(a,file_object)   

# here we close the fileObject
file_object.close()
# we open the file for reading
file_object = open(file_name,'rb')  
# load the object from the file into var b
b = pickle.load(file_object)  

a==b
True
```

In [153]:
image_path = "./data/data_set_simple_2/testing/img12_1.png"
image = Image.open(image_path).convert("L")
img = preprocess_input(image)

In [154]:
classifications = list(
    clf.predict([img])
)

In [155]:
classifications

[90]

### make test

In [156]:
classifications = list(
    clf.predict(test_X)
)

In [157]:
classifications

[90,
 45,
 0,
 135,
 90,
 0,
 0,
 0,
 90,
 45,
 0,
 45,
 90,
 45,
 45,
 90,
 45,
 0,
 135,
 90,
 45,
 0,
 135]

### model accuracy

In [150]:
def get_model_accuracy(model, X, Y):
    classifications = model.predict(X)
    successful = 0
    for (tX, tY) in list(
        zip(Y, classifications)
    ):
        if tX == tY or tX + 15 == tY or tX - 15 == tY:
            successful += 1
    return successful/float(len(Y))

In [151]:
get_model_accuracy(clf, test_X, test_Y)

0.8260869565217391

#### accuracy for color:
85% accurate for testing angles that are exact(0,45,90,135).

67% accuracy for testing angles(15,165,30,60,75,105,120,150) other than has been trained(0,45,90,135) for. 

for grayscale:
90% accurate for testing exact angles
79% accuracy for inexact angles

From this we can see that grayscale takes away a lot of confusion that color brings.